## Line 1 Testing

**Graph Structure Tests:**

- Test that the graph must have a finite number of vertices
- Test that the graph supports both directed and bidirected edges
- Test that all directed edges connect existing vertices
- Test that all bidirected edges connect existing vertices
Test that the algorithm can detect cycles in directed edges
Test that cycle detection returns True for a known cycle (e.g., v1→v2→v3→v1)
Test that cycle detection returns False for acyclic graphs
Test that no directed edge goes from a vertex to itself
Test that no bidirected edge goes from a vertex to itself
Test that graph construction rejects attempts to add self-loops
Test that bidirected edges are symmetric (if vi↔vj exists, then vj↔vi must exist)
Test that adding a bidirected edge automatically creates its symmetric pair
Test that directed edges are asymmetric (cannot have both vi→vj and vj→vi simultaneously)
Test that directed edges allow cycles through intermediates (v1→v2→v3→v1 allowed)
Test that mixed edges are allowed (both vi→vj and vi↔vj can exist simultaneously)

Variable Set Tests (11 tests)

Test that all variable sets (V, Y, W, J) are finite
Test that each variable has a unique identifier within a set
Test that variable sets are frozenset type
Test that frozensets cannot be modified after creation (immutability)
Test that frozensets are hashable (can be used as dict keys or in other sets)
Test that sets are unordered (ordering doesn't affect equality)
Test that algorithm results are independent of input ordering
Test that Y and W are disjoint (Y ∩ W = ∅ is enforced)
Test that overlapping Y and W causes the algorithm to fail at Line 2
Test that J can overlap with Y (J ∩ Y ≠ ∅ is allowed)
Test that J can overlap with W (J ∩ W ≠ ∅ is allowed)

Distribution Tests (8 tests)

Test that all probability values are in valid range [0, 1]
Test that distribution is normalized (sum equals 1 within tolerance)
Test that distribution with negative values fails validation
Test that all realizable configurations have positive probability
Test that distributions with structural zeros are flagged as potential issues
Test that distribution respects independencies encoded in graph (Markov compatibility via σ-separation)
Test that distribution generated from a known graph satisfies Markov property
Test that post-intervention distribution P(V|do(J)) is consistent with mutilated graph G_J̄

Intervention Operator Tests (8 tests) - these seem to be fully covered

Test that do(X) removes all incoming directed edges to X (graph surgery)
Test that edges not involving X as target remain unchanged
Test that bidirected edges (confounders) remain after intervention
Test that intervention operator is idempotent (do(X, X) ≡ do(X))
Test that intervention operator is commutative for disjoint sets (do(X, Y) ≡ do(Y, X))
Test that applying interventions in different orders gives the same graph
Test that empty intervention is identity (do(∅) returns original graph unchanged)
Test that empty intervention preserves distribution (P(Y|do(∅)) = P(Y))




## Line 2 Testing



## Line 3 Testing